In [5]:
import ccxt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

exchange = ccxt.binance()
symbol = 'BTC/USDT'
ohlcv_data = exchange.fetch_ohlcv(symbol, timeframe='1h', limit=1000)  # Fetch 1000 hours of data

df = pd.DataFrame(ohlcv_data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

df['price_change'] = df['close'].pct_change()
df['moving_avg_5'] = df['close'].rolling(window=5).mean()

df.dropna(inplace=True)

X = df[['price_change', 'moving_avg_5']]  # Features
y = np.where(df['close'].shift(-1) > df['close'], 1, 0)  # 1 = buy, 0 = sell (binary classification)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# 9. Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy*100:.2f}%')

prediction = model.predict(np.array([X_test[-1]]))
if prediction > 0.5:
    print('Buy Signal')
else:
    print('Sell Signal')

df[['close']].plot(title=f'{symbol} Closing Price')
plt.show()


ModuleNotFoundError: No module named 'ccxt'